In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import http.client
import requests
import io
import typing
from datetime import datetime, timezone
from dateutil import parser

from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.live.nba.endpoints import boxscore

# Free NBA API (Historical)

https://github.com/swar/nba_api/tree/master/docs/examples

Can get historical data for teams, players, games. Updated quite frequently?

In [54]:
def getTeamData(team: str):
    """ Getter for team data.
    Args:
        team (str): Name of team to get data of.

    Returns:
        dict: Metadata of team selected.
        pd.DataFrame: Dataframe of historical game statistics. 
    """
    
    # Get Details of the team.
    teamsDict = teams.get_teams()
    teamSelected = [x for x in teamsDict if x['full_name'] == team][0]
    # Get game details for the team.
    teamGames = leaguegamefinder.LeagueGameFinder(team_id_nullable = teamSelected['id']).get_data_frames()[0]
    
    return teamSelected, teamGames

In [62]:
def getPlayerData(player: str):
    """ Getter for player data.
    Args:
        player (str): Name of player to get data of.
    
    Returns:
        dict: Metadata of player selected.
        pd.DataFrame: Details of player's games.
    """
    
    # Get details of the players.
    playerDict = players.get_players()
    playerSelected = [x for x in playerDict if x['full_name'] == player][0]
    # Get game details for the player.
    playerGames = playergamelog.PlayerGameLog(player_id = playerSelected['id']).get_data_frames()[0]
    
    return playerSelected, playerGames

## Getting team specific data

In [56]:
teamData, gameData = getTeamData("Golden State Warriors")

In [59]:
teamData

{'id': 1610612744,
 'full_name': 'Golden State Warriors',
 'abbreviation': 'GSW',
 'nickname': 'Warriors',
 'city': 'Golden State',
 'state': 'California',
 'year_founded': 1946}

In [58]:
gameData.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612744,GSW,Golden State Warriors,0022200800,2023-02-04,GSW vs. DAL,W,242,119,...,0.789,5.0,33.0,38.0,33,6,5,14,24,6.0
1,22022,1610612744,GSW,Golden State Warriors,0022200783,2023-02-02,GSW @ DEN,L,241,117,...,0.632,10.0,32.0,42.0,25,4,4,17,23,-17.0
2,22022,1610612744,GSW,Golden State Warriors,0022200774,2023-02-01,GSW @ MIN,L,264,114,...,0.794,9.0,49.0,58.0,21,7,4,17,24,-5.0
3,22022,1610612744,GSW,Golden State Warriors,0022200759,2023-01-30,GSW @ OKC,W,240,128,...,0.800,6.0,38.0,44.0,37,9,4,15,18,8.0
4,22022,1610612744,GSW,Golden State Warriors,0022200740,2023-01-27,GSW vs. TOR,W,241,129,...,0.733,5.0,38.0,43.0,40,5,1,12,18,12.0


In [61]:
gameData.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

## Getting player specific data

In [63]:
playerData, playerGameData = getPlayerData("Russell Westbrook")

In [64]:
playerData

{'id': 201566,
 'full_name': 'Russell Westbrook',
 'first_name': 'Russell',
 'last_name': 'Westbrook',
 'is_active': True}

In [65]:
playerGameData.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22022,201566,0022200798,"FEB 04, 2023",LAL @ NOP,L,23,6,11,0.545,...,2,4,4,0,0,2,1,15,-4,1
1,22022,201566,0022200778,"FEB 02, 2023",LAL @ IND,W,32,2,16,0.125,...,2,3,10,0,0,4,2,10,0,1
2,22022,201566,0022200765,"JAN 31, 2023",LAL @ NYK,W,36,7,13,0.538,...,3,6,8,3,0,1,4,17,-5,1
3,22022,201566,0022200757,"JAN 30, 2023",LAL @ BKN,L,30,5,15,0.333,...,6,8,10,1,1,6,2,17,-10,1
4,22022,201566,0022200749,"JAN 28, 2023",LAL @ BOS,L,25,4,14,0.286,...,2,4,7,1,0,5,4,12,-10,1


In [66]:
playerGameData.columns

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

# Free NBA API (Live)

https://github.com/swar/nba_api/blob/master/docs/examples/LiveData.ipynb

In [77]:
def getScoreboard() -> list:
    """ Prints scoreboard of live games.
    
    Returns:
        list: Games currently ongoing.
    """
    board = scoreboard.ScoreBoard()
    
    # Generate Scoreboard
    f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}"
    print("ScoreBoardDate: " + board.score_board_date)
    games = board.games.get_dict()
    for game in games:
        gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
        print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'],
                       homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))
    
    # Return current games
    return [x['gameId'] for x in games]

In [131]:
def getLiveData(gameId: str):
    """ Get live data for current game.
    
    Returns:
        dict: Current game data in dictionary format.
        pd.DataFrame: Dataframe of home team and away team statistics combined.
    """
    box = boxscore.BoxScore(gameId)
    
    # Data Dictionaries
    datadict = {
        "game" : box.game.get_dict(),                   
        "arena" : box.arena.get_dict(),                 
        "awayteam" : box.away_team.get_dict(),              
        "awayplayers" : box.away_team_player_stats.get_dict(), 
        "awaystats" : box.away_team_stats.get_dict(),        
        "hometeam" : box.home_team.get_dict(),             
        "homeplayers" : box.home_team_player_stats.get_dict(),
        "homestats" : box.home_team_stats.get_dict(),      
        "gamedetails" : box.game_details.get_dict(),      
        "officials" : box.officials.get_dict()}
    
    # Get team data (on home team and away team performances)
    df = dataDict['homestats']['statistics']
    df2 = dataDict['awaystats']['statistics']
    n1, n2 = dataDict['hometeam']['teamName'], dataDict['awayteam']['teamName']
    
    # Combine into a single dataframe.
    teamData = pd.DataFrame([df.values(), df2.values()], columns = df.keys(), 
                           index = [n1, n2])
    
    return datadict, teamData

## Live Scoreboard + getting current games

In [109]:
currGames = getScoreboard()

ScoreBoardDate: 2023-02-05
0022200802: Magic vs. Hornets @ 2023-02-06 02:00:00+08:00
0022200803: Cavaliers vs. Pacers @ 2023-02-06 06:00:00+08:00
0022200804: 76ers vs. Knicks @ 2023-02-06 07:00:00+08:00
0022200805: Raptors vs. Grizzlies @ 2023-02-06 07:00:00+08:00
0022200806: Nuggets vs. Timberwolves @ 2023-02-06 08:00:00+08:00
0022200807: Kings vs. Pelicans @ 2023-02-06 08:00:00+08:00


In [79]:
currGames

['0022200802',
 '0022200803',
 '0022200804',
 '0022200805',
 '0022200806',
 '0022200807']

In [132]:
dataDict, currGameData = getLiveData(currGames[0])

In [133]:
dataDict.keys()

dict_keys(['game', 'arena', 'awayteam', 'awayplayers', 'awaystats', 'hometeam', 'homeplayers', 'homestats', 'gamedetails', 'officials'])

In [134]:
currGameData

,assists,assistsTurnoverRatio,benchPoints,biggestLead,biggestLeadScore,biggestScoringRun,biggestScoringRunScore,blocks,blocksReceived,fastBreakPointsAttempted,...,timeLeading,timesTied,trueShootingAttempts,trueShootingPercentage,turnovers,turnoversTeam,turnoversTotal,twoPointersAttempted,twoPointersMade,twoPointersPercentage
Hornets,28,1.866667,28,10,20-30,10,38-37,6,2,11,...,PT22M54.00S,4,93.12,0.606744,15,0,15,50,33,0.66
Magic,25,2.083333,39,12,73-61,10,38-37,2,6,5,...,PT23M53.00S,4,104.20,0.571017,12,0,12,68,34,0.50
